In [1]:
# Import
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1627403337879_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Create Spark entry point
# spark = SparkSession.builder.enableHiveSupport().appName('Analytical_ETL').getOrCreate()  # Main entry point to Spark cluster
# spark.conf.set("fs.s3.endpoint", "s3-sa-east-1.amazonaws.com")
# sc = spark.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create Spark entry point
spark = SparkSession.builder.appName('Analytical_ETL').getOrCreate()  # Main entry point to Spark cluster
spark.conf.set("fs.s3.endpoint", "s3-sa-east-1.amazonaws.com")
sc = spark.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read and Process Most Recent Trade Data

In [3]:
# Read Parquet Files From AWS S3 bucket
trades_df = spark.read.parquet("s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/All_Ingested_CSV_Files_2/trade/trade_dt2=2020-08-06")
trades_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMA', exchange='NYSE', event_tm='2020-08-06 09:30:00.0', event_seq_nb='10', trade_pr=74.49, max_arrival_tm=datetime.datetime(2020, 8, 6, 10, 49, 37, 345000))

In [4]:
# Create temporary view of parquet files as "trades" table for next step
trades_df.createOrReplaceTempView("trades")

# Create Trade Staging Table --> Update prior script so we can update our files.
tmp_trade_ma_df = spark.sql("select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr from trades where trade_dt = '2020-08-06'")

# Create Spark Temp View
tmp_trade_ma_df.createOrReplaceTempView("tmp_trade_moving_avg")

# Show head
tmp_trade_ma_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMA', exchange='NYSE', event_tm='2020-08-06 09:30:00.0', event_seq_nb='10', trade_pr=74.49)

In [ ]:
# DROP HIVE TABLE IF EXISTS
spark.sql("DROP TABLE IF EXISTS temp_trade_moving_avg")

In [8]:
# Create mov_avg df
mov_avg_df = spark.sql("""
    select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr,
    AVG(trade_pr) OVER (PARTITION BY (symbol)
    ORDER BY CAST(event_tm AS timestamp) 
    RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr    
    from tmp_trade_moving_avg""")

# Save Temp View -> Hive Table for Staging
mov_avg_df.write.saveAsTable("temp_trade_moving_avg")

# Show head
mov_avg_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMA', exchange='NYSE', event_tm='2020-08-06 09:30:00.0', event_seq_nb='10', trade_pr=74.49, mov_avg_pr=76.61399999999999)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [7]:
# One more test

mov_avg_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+--------+--------------------+------------+--------+----------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|trade_pr|mov_avg_pr|
+----------+------+--------+--------------------+------------+--------+----------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 09:30:...|          10|   74.49|    76.614|
|2020-08-06|  SYMA|    NYSE|2020-08-06 09:30:...|         100|   76.31|    76.614|
|2020-08-06|  SYMA|    NYSE|2020-08-06 09:30:...|          90|   75.71|    76.614|
|2020-08-06|  SYMA|    NYSE|2020-08-06 09:30:...|          40|   77.12|    76.614|
|2020-08-06|  SYMA|    NYSE|2020-08-06 09:30:...|          70|   78.23|    76.614|
+----------+------+--------+--------------------+------------+--------+----------+
only showing top 5 rows

# Process Previous Trades Data

In [9]:
# Get Previous Date Value
prev_date_str = "2020-08-05"

# Read previous trades data using Spark
prev_trades_df = spark.read.parquet(f"s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/All_Ingested_CSV_Files_2/trade/trade_dt2={prev_date_str}")

# Show head
prev_trades_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-05', symbol='SYMA', exchange='NYSE', event_tm='2020-08-05 09:30:00.0', event_seq_nb='100', trade_pr=77.79, max_arrival_tm=datetime.datetime(2020, 8, 5, 21, 30, 19, 773000))

In [10]:
# Create temporary view of parquet files as "trades" table for next step
prev_trades_df.createOrReplaceTempView("prev_trades")

# Use Spark to read trade table w/ date partition (trade_dt="2020-07-28") --> This is how we filter for "last" trade price.
tmp_last_trade_df = spark.sql(f"select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr from prev_trades where trade_dt = '{prev_date_str}'")

# Show head
tmp_last_trade_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-05', symbol='SYMA', exchange='NYSE', event_tm='2020-08-05 09:30:00.0', event_seq_nb='100', trade_pr=77.79)

In [11]:
# Create Spark Temporary View
tmp_last_trade_df.createOrReplaceTempView("tmp_last_trade")

# Calculate Last Trade Price Using Spark Temp View
last_trade_ma_df = spark.sql("""select a.trade_dt, a.symbol, a.exchange, a.event_tm, 
a.event_seq_nb, a.last_pr from 
(select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr,
AVG(trade_pr)
OVER (PARTITION BY (symbol)
ORDER BY CAST(event_tm as timestamp) 
RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as last_pr
FROM tmp_trade_moving_avg) a
""")

# Save Temp View Into Hive Table For Staging
last_trade_ma_df.write.saveAsTable("temp_last_trade")

# Show head
last_trade_ma_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMA', exchange='NYSE', event_tm='2020-08-06 09:30:00.0', event_seq_nb='30', last_pr=76.614)

# Populate The Latest Trade and Latest Moving Average Trade Price To The Quote Records

In [12]:
# First, create temporary view of our data.
quotes_df = spark.read.parquet("s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/All_Ingested_CSV_Files_2/quote/trade_dt2=2020-08-06")
# Note that I have quote_dt, even though it should be trade_dt.
quotes_df.createOrReplaceTempView("quotes")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
quotes_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMC', exchange='NYSE', event_tm='2020-08-06 09:30:00.0', event_seq_nb='13', trade_pr='-', max_arrival_tm=datetime.datetime(2020, 8, 6, 11, 2, 58, 675000), rec_type='Q', arrival_tm='2020-08-06 11:02:58.675', bid_pr='158.03838165179127', bid_size='100', ask_pr='158.5695078200935', ask_size='100', execution_id='NA', trade_size='NA')

In [15]:
# Test and show quotes_df
# quotes_df_all = spark.sql("""SELECT * FROM quotes""")
# quotes_df_all.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Test and show all from temp_trade_moving_avg
# quotes_df_all = spark.sql("""SELECT * FROM temp_trade_moving_avg""")
# quotes_df_all.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# I need to re-do Step 2 s.t. I have all of my necessary data so I can take bid_pr, ..., ask_size.

In [35]:
quotes_df2 = spark.sql("""
SELECT bid_pr, trade_pr FROM quotes
""")
quotes_df2.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(bid_pr='158.03838165179127', trade_pr='-')

In [15]:
# Now, create Spark SQL statement to union data from both temp views of data

quote_union = spark.sql("""
SELECT trade_dt,rec_type,symbol,event_tm,event_seq_nb,exchange,bid_pr,bid_size,ask_pr,
ask_size,null as trade_pr,null as mov_avg_pr FROM quotes
UNION ALL
SELECT trade_dt, "T" as rec_type,symbol,event_tm,event_seq_nb,exchange,null as bid_pr,null as bid_size,null as ask_pr,
null as ask_size,trade_pr,mov_avg_pr FROM temp_trade_moving_avg
""")

quote_union.createOrReplaceTempView("quote_union")

# Show df
quote_union.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+------+--------------------+------------+--------+------------------+--------+------------------+--------+--------+----------+
|  trade_dt|rec_type|symbol|            event_tm|event_seq_nb|exchange|            bid_pr|bid_size|            ask_pr|ask_size|trade_pr|mov_avg_pr|
+----------+--------+------+--------------------+------------+--------+------------------+--------+------------------+--------+--------+----------+
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          13|    NYSE|158.03838165179127|     100| 158.5695078200935|     100|    null|      null|
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          32|    NYSE|157.71166149994355|     100|159.34694545037547|     100|    null|      null|
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          99|    NYSE|159.50664417328284|     100|  159.708657615948|     100|    null|      null|
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          64|    NYSE|159.68220039818317|     100|160.064634048

In [42]:
# Quote Union Test 1
test_quote_union_df1 = spark.sql("""
SELECT trade_dt,rec_type,symbol,event_tm,event_seq_nb,exchange,bid_pr,bid_size,ask_pr,
ask_size,null as trade_pr,null as mov_avg_pr FROM quotes""")
test_quote_union_df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+------+--------------------+------------+--------+------------------+--------+------------------+--------+--------+----------+
|  trade_dt|rec_type|symbol|            event_tm|event_seq_nb|exchange|            bid_pr|bid_size|            ask_pr|ask_size|trade_pr|mov_avg_pr|
+----------+--------+------+--------------------+------------+--------+------------------+--------+------------------+--------+--------+----------+
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          13|    NYSE|158.03838165179127|     100| 158.5695078200935|     100|    null|      null|
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          32|    NYSE|157.71166149994355|     100|159.34694545037547|     100|    null|      null|
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          99|    NYSE|159.50664417328284|     100|  159.708657615948|     100|    null|      null|
|2020-08-06|       Q|  SYMC|2020-08-06 09:30:...|          64|    NYSE|159.68220039818317|     100|160.064634048

In [41]:
# Quote Union Test 2

test_quote_union_df2 = spark.sql("""SELECT trade_dt,"T" as rec_type,symbol,event_tm,event_seq_nb,exchange,null as bid_pr,null as bid_size,null as ask_pr,
null as ask_size,trade_pr,mov_avg_pr FROM temp_trade_moving_avg""")
test_quote_union_df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
|  trade_dt|rec_type|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|mov_avg_pr|
+----------+--------+------+--------------------+------------+--------+------+--------+------+--------+--------+----------+
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          10|    NYSE|  null|    null|  null|    null|   74.49|    76.614|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|         100|    NYSE|  null|    null|  null|    null|   76.31|    76.614|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          90|    NYSE|  null|    null|  null|    null|   75.71|    76.614|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          40|    NYSE|  null|    null|  null|    null|   77.12|    76.614|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          70|    NYSE|  null|    null|  null|    null|   78.23|    76.614|
|2020-08

In [14]:
# One more time union test
test_quote_union_df3 = spark.sql("""
SELECT trade_dt,"T" as rec_type,symbol,event_tm,event_seq_nb,exchange,null as bid_pr,null as bid_size,null as ask_pr,
null as ask_size,trade_pr,mov_avg_pr FROM temp_trade_moving_avg
UNION ALL
SELECT trade_dt,rec_type,symbol,event_tm,event_seq_nb,exchange,bid_pr,bid_size,ask_pr,
ask_size,null as trade_pr,null as mov_avg_pr FROM quotes
""")
test_quote_union_df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+------+--------------------+------------+--------+------+--------+------+--------+--------+-----------------+
|  trade_dt|rec_type|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|trade_pr|       mov_avg_pr|
+----------+--------+------+--------------------+------------+--------+------+--------+------+--------+--------+-----------------+
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          30|    NYSE|  null|    null|  null|    null|    76.9|76.61399999999999|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          10|    NYSE|  null|    null|  null|    null|   74.49|76.61399999999999|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|         100|    NYSE|  null|    null|  null|    null|   76.31|76.61399999999999|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          90|    NYSE|  null|    null|  null|    null|   75.71|76.61399999999999|
|2020-08-06|       T|  SYMA|2020-08-06 09:30:...|          40|    NYSE|  null|    n

In [ ]:
# test_quote_union_df3.select("trade_pr").distinct().collect()

In [16]:
# Populate The Latest trade_pr and mov_avg_pr

quote_union_update = spark.sql("""
select *,
last_value(trade_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_trade_pr,
last_value(mov_avg_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_mov_avg_pr
from quote_union
""")

quote_union_update.createOrReplaceTempView("quote_union_update")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
quote_union_update.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', rec_type='Q', symbol='SYMB', event_tm='2020-08-06 09:30:00.0', event_seq_nb='4', exchange='NYSE', bid_pr='35.80141900363462', bid_size='100', ask_pr='36.8613695404052', ask_size='100', trade_pr=None, mov_avg_pr=None, last_trade_pr=None, last_mov_avg_pr=None)

In [18]:
# Filter For Quote Records
quote_update = spark.sql("""
    select trade_dt, symbol, event_tm, event_seq_nb, exchange,
    bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr
    from quote_union_update
    where rec_type = 'Q'
""")

# Create temporary view of data
quote_update.createOrReplaceTempView("quote_update")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Try subquery

# test_quote_data = spark.sql("""
#     select q.trade_dt, q.symbol, q.event_tm, q.event_seq_nb, q.exchange,
#     q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, last_trade_pr, last_mov_avg_pr
#     from quote_union_update q
#     RIGHT JOIN

#     (select trade_dt, symbol, event_tm, event_seq_nb, exchange,
#     bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr
#     from quote_union_update
#     where rec_type = 'T' and symbol = "SYMC"
# """)
# test_quote_data.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMC', event_tm='2020-08-06 09:30:00.0', event_seq_nb='30', exchange='NYSE', bid_pr=None, bid_size=None, ask_pr=None, ask_size=None, last_trade_pr=158.06, last_mov_avg_pr=159.442)

In [19]:
quote_update.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(trade_dt='2020-08-06', symbol='SYMB', event_tm='2020-08-06 09:30:00.0', event_seq_nb='68', exchange='NYSE', bid_pr='35.805793618254164', bid_size='100', ask_pr='36.16367101821172', ask_size='100', last_trade_pr=None, last_mov_avg_pr=None)

# Maybe, it shows null for trade_pr for first 20 rows because we are partitioning by symbol?

In [23]:
# Apply broadcast join --> May need to apply data type casting to bid_pr, ..., ask_size, etc.
quote_final = spark.sql("""
    select trade_dt, symbol, event_tm, event_seq_nb, exchange,
    bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr,
    bid_pr - close_pr as bid_pr_mv,
    ask_pr - close_pr as ask_pr_mv
    from (
    
    select /* + BROADCAST(t) */
    q.trade_dt, q.symbol, q.event_tm, q.event_seq_nb, q.exchange,
    q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, q.last_trade_pr,
    q.last_mov_avg_pr,
    t.last_pr as close_pr
    from quote_update q
    left outer join temp_last_trade t on
    (q.symbol = t.symbol and q.exchange = t.exchange))
""")

# Show us the data so we can pick which trade_date to use
quote_final.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+--------------------+------------+--------+------------------+--------+-----------------+--------+-------------+---------------+--------------------+-------------------+
|  trade_dt|symbol|            event_tm|event_seq_nb|exchange|            bid_pr|bid_size|           ask_pr|ask_size|last_trade_pr|last_mov_avg_pr|           bid_pr_mv|          ask_pr_mv|
+----------+------+--------------------+------------+--------+------------------+--------+-----------------+--------+-------------+---------------+--------------------+-------------------+
|2020-08-06|  SYMB|2020-08-06 09:30:...|          24|    NYSE|33.072382405139976|     100|34.09166855576172|     100|         null|           null| -0.8216175948600224|0.19766855576172304|
|2020-08-06|  SYMB|2020-08-06 09:30:...|          24|    NYSE|33.072382405139976|     100|34.09166855576172|     100|         null|           null| -0.8216175948600224|0.19766855576172304|
|2020-08-06|  SYMB|2020-08-06 09:30:...|          24|  

In [26]:
# Write The Final DF Into Azure Blob Storage At Corresponding Partition
trade_date = "2020-08-06"
quote_final.write.mode("overwrite").parquet(f"s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/All_Ingested_CSV_Files_2/quote-trade-analytical/trade_dt={trade_date}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Stop Spark Cluster
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Congrats! We are now almost finished with our Guided Capstone.

### Step 5 will include automating our data pipeline from end-to-end while using bash scripting.
### Check S3 bucket to ensure we have the correct files.